In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql import functions as F
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, lead, mean

In [2]:
fpath = "/mnt/compressordata/compressors_eagleford_travis_20181025142853.txt"
df = spark.read.format("csv").option("header", "true").option("delimiter","\t").option("inferSchema", "true").load(fpath)

In [3]:
#hourwindow = Window.partitionBy('Asset Name').orderBy('Local Timestamp').rowsBetween(-10, 0)
#moving_avg = mean(df['Compressor Oil Pressure']).over(hourwindow)
#df = df.withColumn('moving_avg', moving_avg)

#windowSpec = Window.orderBy(F.col("Local Timestamp").partitionBy).rangeBetween(-days(3), 0)
#window = Window.orderBy("date").partitionBy("country").rangeBetween(-3, 3)



#create window by casting timestamp to long (number of seconds)
#w = (Window.orderBy(F.col("Local Timestamp").cast('long')).rangeBetween(-days(7), 0))

#df = df.withColumn('rolling_average', F.avg("dollars").over(w))

#trial = []
#trialmin = df.groupby("Asset Name").agg(F.max("Fuel Pressure"), F.min("Fuel Pressure"))
#display(trialmin)

df.createOrReplaceTempView("dataframe")
df = spark.sql("SELECT *, lag(`SD Status Code`, 10) over (order by `Asset Name`,`Local Timestamp`) AS futurestatus FROM dataframe")

df=df.na.fill(0)
dataset = df

In [4]:
display(df)

Id,Asset Name,Local Timestamp,UTC Milliseconds,Compressor Oil Pressure,Compressor Oil Temp,Compressor Stages,Cylinder 1 Discharge Temp,Cylinder 2 Discharge Temp,Cylinder 3 Discharge Temp,Cylinder 4 Discharge Temp,Downtime Hrs Yest,Engine Oil Pressure,Engine Oil Temp,Facility Desc,Facility ID,Fuel Pressure,Gas Flow Rate,Gas Flow Rate_RAW,Horsepower,Last Successful Comm Time,Max Discharge Pressure,Max Gas Flowrate,Max RPMs,Max Suction Pressure,Pct Successful Msgs Today,RPM,Run Status,Runtime Hrs,SD Status Code,Stage 1 Discharge Pressure,Stage 2 Discharge Pressure,Stage 3 Discharge Pressure,Successful Msgs Today,Suction Pressure,Suction Temp,TOW Comp Name,Unit Size,futurestatus
96108,"Baker, A.J. A 2 CMP",10/1/2017 10:00:01.5320129 AM,1506870001532,45.0,170.4998,3,244.0,231.0,218.0,0.0,0.743118,0,103.0,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,577.4378,577.4378,203,10/01 09:00:00,1000,886,1800,50,100,1464,1-RUNNING,22136.0,NO FAULT PRESENT,167.0,409.0,942.0,1623,357.0,83.0,A.J. BAKER A #2H,3306TA,null
96806,"Baker, A.J. A 2 CMP",10/1/2017 10:00:01.8480072 PM,1506913201848,47.0,171.0087,3,241.0,229.5043,216.0,0.0,0.3350022,0,87.19855,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,571.8575,571.8575,203,10/01 21:00:01,1000,886,1800,50,100,1420,1-RUNNING,22148.0,NO FAULT PRESENT,168.0,402.0,936.0,3783,359.0,79.0,A.J. BAKER A #2H,3306TA,null
96807,"Baker, A.J. A 2 CMP",10/1/2017 10:01:00.879013 PM,1506913260879,51.0,172.0,3,243.0,230.0,218.0,0.0,0.3344445,0,87.10007,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,572.4468,572.4468,203,10/01 21:01:00,1000,886,1800,50,100,1448,1-RUNNING,22148.0,NO FAULT PRESENT,167.0,405.0,928.0,3786,354.0,78.0,A.J. BAKER A #2H,3306TA,null
96109,"Baker, A.J. A 2 CMP",10/1/2017 10:01:01.6590118 AM,1506870061659,41.0,171.0,3,246.0,232.0,218.0,0.0,0.74255,0,103.2002,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,576.7263,576.7263,203,10/01 09:01:00,1000,886,1800,50,100,1451,1-RUNNING,22136.0,NO FAULT PRESENT,168.0,397.0,941.0,1626,361.0,81.99909,A.J. BAKER A #2H,3306TA,null
96808,"Baker, A.J. A 2 CMP",10/1/2017 10:02:00.8700103 PM,1506913320870,50.0,171.6669,3,243.4999,229.5004,217.0,0.0,0.3338777,0,87.0,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,573.0457,573.0457,203,10/01 21:02:00,1000,886,1800,50,100,1455,1-RUNNING,22148.0,NO FAULT PRESENT,168.0,403.0004,927.0,3789,361.0,77.80016,A.J. BAKER A #2H,3306TA,null
96110,"Baker, A.J. A 2 CMP",10/1/2017 10:02:01.6770019 AM,1506870121677,47.0,169.0,3,244.0,231.0,218.0,0.0,0.7419831,0,103.4,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,576.0161,576.0161,203,10/01 09:02:00,1000,886,1800,50,100,1461,1-RUNNING,22136.0,NO FAULT PRESENT,167.0,401.0,948.0,1629,355.0,81.0,A.J. BAKER A #2H,3306TA,null
96809,"Baker, A.J. A 2 CMP",10/1/2017 10:03:00.8840026 PM,1506913380884,40.0,171.3337,3,244.0,229.0005,216.6669,0.0,0.3333108,0,86.95652,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,573.6448,573.6448,203,10/01 21:03:00,1000,886,1800,50,100,1452,1-RUNNING,22148.0,NO FAULT PRESENT,167.0,401.0,950.0,3792,352.0,77.60025,A.J. BAKER A #2H,3306TA,null
96111,"Baker, A.J. A 2 CMP",10/1/2017 10:03:01.7570037 AM,1506870181757,44.0,170.0,3,246.0,231.3333,218.0,0.0,0.7414154,0,103.6,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,575.3051,575.3051,203,10/01 09:03:00,1000,886,1800,50,100,1465,1-RUNNING,22136.0,NO FAULT PRESENT,167.4999,398.0,961.0,1632,355.4,81.20002,A.J. BAKER A #2H,3306TA,null
96810,"Baker, A.J. A 2 CMP",10/1/2017 10:04:00.9750061 PM,1506913440975,46.0,171.0,3,241.0,228.5001,216.3333,0.0,0.3327431,0,86.91298,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,574.2448,574.2448,203,10/01 21:04:00,1000,886,1800,50,100,1445,1-RUNNING,22148.0,NO FAULT PRESENT,168.0,411.0,939.0,3795,355.0,77.40009,A.J. BAKER A #2H,3306TA,null
96112,"Baker, A.J. A 2 CMP",10/1/2017 10:04:01.8660125 AM,1506870241866,50.0,171.0,3,244.0,231.6668,218.0,0.0,0.7408476,0,103.8001,"Baker, A.J. A 2 CMP",BAKER_AJ_2H_CMP,null,574.5938,574.5938,203,10/01 09:04:01,1000,886,1800,50,100,1462,1-RUNNING,22136.0,NO FAULT PRESENT,168.0,408.0

In [5]:
categoricalColumns = ["Asset Name", "Run Status"]

numericCols = ["Compressor Oil Pressure" , "Compressor Oil Temp", 
               "Engine Oil Temp", "Suction Pressure", "Suction Temp"]

In [6]:
#asemblerInputs required for feature transformer that merges multiple columns to vector column

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
all_columns = categoricalColumns + numericCols
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index", handleInvalid="skip")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
    
label_stringIdx = StringIndexer(inputCol="futurestatus", outputCol="label",handleInvalid="skip")
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [label_stringIdx, assembler]

pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(dataset)
result = pipelineModel.transform(dataset)
selectedcols = ["label", "features"] + all_columns
dataset = result.select(selectedcols)

In [7]:
splits = dataset.randomSplit([.65, .35], seed = 100)
trainingData = splits[0]
testData = splits[1]

In [8]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees = 4)

In [9]:
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)

In [10]:
selected = predictions.select("label", "rawPrediction", "prediction", "probability")
#display(selected)

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
rmse = evaluator.evaluate(predictions)
print(1-rmse)

0.115917015432

In [12]:
rfModel.save('/FileStore/mymodel')